In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
from datetime import timedelta
import io
import requests
import re
from bs4 import BeautifulSoup
from functools import partial, reduce
from download_data import get_url, clean_dataframe, get_csv
from transform_data import *
import os
import glob
import boto3
from to_csv_on_s3 import to_csv_on_s3

## 1-2. パラメーターの設定
# 元データをダウンロードするためのフォルダ
# reading s3 bucket
client = boto3.client('s3')

dirname = os.getcwd()
# FOLDER_RAW = os.path.join(dirname, '元データ/')
BUCKET_NAME = 'jpx-future-bucket'
FOLDER_RAW = "raw_data/"

# 完成したデータを保存するためのフォルダ
# FOLDER_FINAL = os.path.join(dirname, '完成データ/')
# FOLDER_FINAL = 's3://jpx-future-bucket/完成データ/'
FOLDER_FINAL = "final_data/"

# 企業のグループ分けに使うExcelファイル
PATH_GROUP = os.path.join(dirname, '企業名グループ.xlsx')

# TOPIXに掛ける係数（225合計と合わせて総合計を計算する際に使う）
TOPIX_MULTI = 1.0

In [2]:
days = 0
print("days", days)
print("TOPIX_MULTI", TOPIX_MULTI)

day = dt.today() - timedelta(days=days)
colnames = ['institutions_sell_code', 'institutions_sell', 
        'institutions_sell_eng', 'volume_sell', 'institutions_buy_code', 
        'institutions_buy', 'institutions_buy_eng', 'volume_buy']

print("2. データのダウンロード")

### 2-2. 日中立会取引データのダウンロード
url_wholeday = get_url(datatype=3, day=day)
if url_wholeday == "No data":
    # 休日などでファイルがない場合はエラーメッセージを返します
    text = "%s年%s月%s日の日中立会取引データが見つかりません"%(day.year, day.month, day.day)
    print(text)
elif url_wholeday == "No page":
    text = "メインページが見つかりません。https://www.jpx.co.jp/markets/derivatives/participant-volume/index.html　を確認してください"
    print(text)
elif requests.get(url_wholeday).ok:
    s=requests.get(url_wholeday).content
    df_wholeday = clean_dataframe(get_csv(s, colnames = colnames), day = day)
    filename = FOLDER_RAW + "日中立会取引" + str(day.year) +"-"+ str(day.month) +"-"+ str(day.day) + ".csv"
    # df_wholeday.to_csv(filename)
    to_csv_on_s3(df_wholeday, bucketName=BUCKET_NAME, fileName=filename)
    text = "%s年%s月%s日の日中立会取引データをダウンロードしました"%(day.year, day.month, day.day)
    print(text)
else:
    text = "何らかの理由でデータが見つかりません"
    print(text)

### 2-3. 日中JNET取引データのダウンロード
url_wholeday_JNET = get_url(datatype=4, day=day)
if url_wholeday == "No data":
    # 休日などでファイルがない場合はエラーメッセージを返します
    text = "%s年%s月%s日の日中JNET取引データが見つかりません"%(day.year, day.month, day.day)
    print(text)
elif url_wholeday == "No page":
    text = "メインページが見つかりません。https://www.jpx.co.jp/markets/derivatives/participant-volume/index.html　を確認してください"
    print(text)
elif requests.get(url_wholeday_JNET).ok:
    s=requests.get(url_wholeday_JNET).content
    df_wholeday_JNET = clean_dataframe(get_csv(s, colnames = colnames), day = day)
    filename = FOLDER_RAW + "日中JNET取引" + str(day.year) +"-"+ str(day.month) +"-"+ str(day.day) + ".csv"
    # df_wholeday_JNET.to_csv(filename)
    to_csv_on_s3(df_wholeday_JNET, bucketName=BUCKET_NAME, fileName=filename)
    text = "%s年%s月%s日の日中JNET取引データをダウンロードしました"%(day.year, day.month, day.day)
    print(text)
else:
    text = "何らかの理由でデータが見つかりません"
    print(text)

### 2-4. ナイト立会取引データのダウンロード
url_night = get_url(datatype=1, day=day)
if url_wholeday == "No data":
    # 休日などでファイルがない場合はエラーメッセージを返します
    text = "%s年%s月%s日のナイト立会取引データが見つかりません"%(day.year, day.month, day.day)
    print(text)
elif url_wholeday == "No page":
    text = "メインページが見つかりません。https://www.jpx.co.jp/markets/derivatives/participant-volume/index.html　を確認してください"
    print(text)
elif requests.get(url_night).ok:
    s=requests.get(url_night).content
    df_night = clean_dataframe(get_csv(s, colnames = colnames), day = day)
    filename = FOLDER_RAW + "ナイト立会取引" + str(day.year) +"-"+ str(day.month) +"-"+ str(day.day) + ".csv"
    # df_night.to_csv(filename)
    to_csv_on_s3(df_night, bucketName=BUCKET_NAME, fileName=filename)
    text = "%s年%s月%s日のナイト立会取引データをダウンロードしました"%(day.year, day.month, day.day)
    print(text)
else:
    text = "何らかの理由でデータが見つかりません"
    print(text)

### 2-5. ナイトJNET取引データのダウンロード
url_night_JNET = get_url(datatype=2, day=day)
if url_wholeday == "No data":
    # 休日などでファイルがない場合はエラーメッセージを返します
    text = "%s年%s月%s日のナイトJNET取引データが見つかりません"%(day.year, day.month, day.day)
    print(text)
elif url_wholeday == "No page":
    text = "メインページが見つかりません。https://www.jpx.co.jp/markets/derivatives/participant-volume/index.html　を確認してください"
    print(text)
elif requests.get(url_night_JNET).ok:
    s=requests.get(url_night_JNET).content
    df_night_JNET = clean_dataframe(get_csv(s, colnames = colnames), day = day)
    filename = FOLDER_RAW + "ナイトJNET取引" + str(day.year) +"-"+ str(day.month) +"-"+ str(day.day) + ".csv"
    # df_night_JNET.to_csv(filename)
    to_csv_on_s3(df_night_JNET, bucketName=BUCKET_NAME, fileName=filename)
    text = "%s年%s月%s日のナイトJNET取引データをダウンロードしました"%(day.year, day.month, day.day)
    print(text)
else:
    text = "何らかの理由でデータが見つかりません"
    print(text)

days 0
TOPIX_MULTI 1.0
2. データのダウンロード
2021年1月27日の日中立会取引データをダウンロードしました
2021年1月27日の日中JNET取引データをダウンロードしました
2021年1月27日のナイト立会取引データをダウンロードしました
2021年1月27日のナイトJNET取引データをダウンロードしました


In [3]:
df_wholeday.head()

,institutions_sell_code,institutions_sell,institutions_sell_eng,volume_sell,institutions_buy_code,institutions_buy,institutions_buy_eng,volume_buy,JPX_code,instrument,date
2,11788,ソシエテ・ジェネラル,Societe Generale,11383,12479,ＡＢＮアムロクリア,ABN AMRO Clearing,9821,166030005,FUT_TOPIX_2103,2021-01-27
3,12479,ＡＢＮアムロクリア,ABN AMRO Clearing,10069,11788,ソシエテ・ジェネラル,Societe Generale,9656,166030005,FUT_TOPIX_2103,2021-01-27
4,11560,ゴールドマンサックス,Goldman Sachs Japan,2071,12792,BofA証券,BofA Securities,2084,166030005,FUT_TOPIX_2103,2021-01-27
5,12400,野村證券,Nomura Securities,1257,12800,モルガンスタンレー,Morgan Stanley MUFG,1878,166030005,FUT_TOPIX_2103,2021-01-27
6,12800,モルガンスタンレー,Morgan Stanley MUFG,1172,11714,ＪＰモルガン証券,JPMorgan Japan,1578,166030005,FUT_TOPIX_2103,2021-01-27


In [4]:
## 3. データの初期整理
text = "\n3. データの初期整理"
print(text)
### 3-1. 先物/OP識別カラムを加える
df_wholeday["先物/OP"] = np.nan
df_wholeday["先物/OP"] = df_wholeday.JPX_code.apply(fut_or_op)

df_wholeday_JNET["先物/OP"] = np.nan
df_wholeday_JNET["先物/OP"] = df_wholeday_JNET.JPX_code.apply(fut_or_op)

df_night["先物/OP"] = np.nan
df_night["先物/OP"] = df_night.JPX_code.apply(fut_or_op)

df_night_JNET["先物/OP"] = np.nan
df_night_JNET["先物/OP"] = df_night_JNET.JPX_code.apply(fut_or_op)

## 3-2. 銘柄を抽出する
df_wholeday["銘柄"] = np.nan
df_wholeday["銘柄"] = df_wholeday.JPX_code.apply(get_meigara)

df_wholeday_JNET["銘柄"] = np.nan
df_wholeday_JNET["銘柄"] = df_wholeday_JNET.JPX_code.apply(get_meigara)

df_night["銘柄"] = np.nan
df_night["銘柄"] = df_night.JPX_code.apply(get_meigara)

df_night_JNET["銘柄"] = np.nan
df_night_JNET["銘柄"] = df_night_JNET.JPX_code.apply(get_meigara)


3. データの初期整理


In [5]:
df_wholeday.head()

,institutions_sell_code,institutions_sell,institutions_sell_eng,volume_sell,institutions_buy_code,institutions_buy,institutions_buy_eng,volume_buy,JPX_code,instrument,date,先物/OP,銘柄
2,11788,ソシエテ・ジェネラル,Societe Generale,11383,12479,ＡＢＮアムロクリア,ABN AMRO Clearing,9821,166030005,FUT_TOPIX_2103,2021-01-27,先物,TOPIX
3,12479,ＡＢＮアムロクリア,ABN AMRO Clearing,10069,11788,ソシエテ・ジェネラル,Societe Generale,9656,166030005,FUT_TOPIX_2103,2021-01-27,先物,TOPIX
4,11560,ゴールドマンサックス,Goldman Sachs Japan,2071,12792,BofA証券,BofA Securities,2084,166030005,FUT_TOPIX_2103,2021-01-27,先物,TOPIX
5,12400,野村證券,Nomura Securities,1257,12800,モルガンスタンレー,Morgan Stanley MUFG,1878,166030005,FUT_TOPIX_2103,2021-01-27,先物,TOPIX
6,12800,モルガンスタンレー,Morgan Stanley MUFG,1172,11714,ＪＰモルガン証券,JPMorgan Japan,1578,166030005,FUT_TOPIX_2103,2021-01-27,先物,TOPIX


In [6]:
    ## 4. 立会とJNETを統合し、限月ごとに整理して並び替える
    text = "\n4. 立会とJNETを統合します"
    print(text)
    ### 4-2. 日中の立会とJNETを統合する

    #　日中立会
    text = "4-2. 日中取引の統合…"
    print(text)
    # 225ラージ
    df_wholeday_large = df_wholeday[(df_wholeday["銘柄"]=="日経225（ラージ）") & (df_wholeday["先物/OP"] == "先物")]

    # 限月の抽出
    df_wholeday_large = get_gengetsu(df_wholeday_large)

    # サイズの確認
    for g in df_wholeday_large["限月"].unique():
        shape = df_wholeday_large[df_wholeday_large["限月"] == g].shape
        
        text = f"日中225ラージの第{g}限月のデータサイズは{shape}"
        print(text)
    
    #　JNET
    df_wholeday_JNET_large = df_wholeday_JNET[(df_wholeday_JNET["銘柄"]=="日経225（ラージ）") & (df_wholeday_JNET["先物/OP"] == "先物")]

    # 限月の抽出
    df_wholeday_JNET_large = get_gengetsu(df_wholeday_JNET_large)

    # サイズの確認
    for g in df_wholeday_JNET_large["限月"].unique():
        shape = df_wholeday_JNET_large[df_wholeday_JNET_large["限月"] == g].shape
        
        text = f"日中225ラージJNETの第{g}限月のデータサイズは{shape}"
        print(text)
    
    df_wholeday_large_stacked = make_long_df(df_wholeday_large)
    df_wholeday_JNET_large_stacked = make_long_df(df_wholeday_JNET_large)
    df_wholeday_large_sum = merge_JNET(df_wholeday_large_stacked, df_wholeday_JNET_large_stacked)

    # 225ミニ
    df_wholeday_mini = df_wholeday[(df_wholeday["銘柄"]=="日経225（ミニ）") & (df_wholeday["先物/OP"] == "先物")]

    # 限月の抽出
    df_wholeday_mini = get_gengetsu(df_wholeday_mini)

    # サイズの確認
    for g in df_wholeday_mini["限月"].unique():
        shape = df_wholeday_mini[df_wholeday_mini["限月"] == g].shape
        
        text = f"日中225ミニの第{g}限月のデータサイズは{shape}"
        print(text)
    
    # 225ミニJNET
    df_wholeday_JNET_mini = df_wholeday_JNET[(df_wholeday_JNET["銘柄"]=="日経225（ミニ）") & (df_wholeday_JNET["先物/OP"] == "先物")]

    # 限月の抽出
    df_wholeday_JNET_mini = get_gengetsu(df_wholeday_JNET_mini)

    # サイズの確認
    for g in df_wholeday_JNET_mini["限月"].unique():
        shape = df_wholeday_JNET_mini[df_wholeday_JNET_mini["限月"] == g].shape
        
        text = f"日中225ミニJNETの第{g}限月のデータサイズは{shape}"
        print(text)

    df_wholeday_mini_stacked = make_long_df(df_wholeday_mini)
    df_wholeday_JNET_mini_stacked = make_long_df(df_wholeday_JNET_mini)
    df_wholeday_mini_sum = merge_JNET(df_wholeday_mini_stacked, df_wholeday_JNET_mini_stacked)

    # TOPIX
    df_wholeday_topix = df_wholeday[(df_wholeday["銘柄"]=="TOPIX") & (df_wholeday["先物/OP"] == "先物")]

    # 限月の抽出
    df_wholeday_topix = get_gengetsu(df_wholeday_topix)

    # サイズの確認
    for g in df_wholeday_topix["限月"].unique():
        shape = df_wholeday_topix[df_wholeday_topix["限月"] == g].shape
        
        text = f"日中TOPIXの第{g}限月のデータサイズは{shape}"
        print(text)

    df_wholeday_JNET_topix = df_wholeday_JNET[(df_wholeday_JNET["銘柄"]=="TOPIX") & (df_wholeday_JNET["先物/OP"] == "先物")]

    # 限月の抽出
    df_wholeday_JNET_topix = get_gengetsu(df_wholeday_JNET_topix)

    # サイズの確認
    for g in df_wholeday_JNET_topix["限月"].unique():
        shape = df_wholeday_JNET_topix[df_wholeday_JNET_topix["限月"] == g].shape
        
        text = f"日中TOPIX、JNETの第{g}限月のデータサイズは{shape}"
        print(text)
    
    df_wholeday_topix_stacked = make_long_df(df_wholeday_topix)
    df_wholeday_JNET_topix_stacked = make_long_df(df_wholeday_JNET_topix)
    df_wholeday_topix_sum = merge_JNET(df_wholeday_topix_stacked, df_wholeday_JNET_topix_stacked)

    ### 4-3. ナイトセッションの立会とJNETを統合する
    print("4-3. ナイトセッションの統合…")
    #　ナイト立会
    df_night_large = df_night[(df_night["銘柄"]=="日経225（ラージ）") & (df_night["先物/OP"] == "先物")]

    # 限月の抽出
    df_night_large = get_gengetsu(df_night_large)

    # サイズの確認
    for g in df_night_large["限月"].unique():
        shape = df_night_large[df_night_large["限月"] == g].shape
        
        text = f"ナイト日経225ラージの第{g}限月のデータサイズは{shape}"
        print(text)
    
    #　ナイトJNET
    df_night_JNET_large = df_night_JNET[(df_night_JNET["銘柄"]=="日経225（ラージ）") & (df_night_JNET["先物/OP"] == "先物")]

    # 限月の抽出
    df_night_JNET_large = get_gengetsu(df_night_JNET_large)

    # サイズの確認
    for g in df_night_JNET_large["限月"].unique():
        shape = df_night_JNET_large[df_night_JNET_large["限月"] == g].shape
        
        text = f"ナイト日経225ラージJNETの第{g}限月のデータサイズは{shape}"
        print(text)
    
    df_night_large_stacked = make_long_df(df_night_large)
    df_night_JNET_large_stacked = make_long_df(df_night_JNET_large)
    df_night_large_sum = merge_JNET(df_night_large_stacked, df_night_JNET_large_stacked)

    # 225ミニ
    #　ナイト立会
    df_night_mini = df_night[(df_night["銘柄"]=="日経225（ミニ）") & (df_night["先物/OP"] == "先物")]

    # 限月の抽出
    df_night_mini = get_gengetsu(df_night_mini)

    # サイズの確認
    for g in df_night_mini["限月"].unique():
        shape = df_night_mini[df_night_mini["限月"] == g].shape
        
        text = f"ナイト日経225ミニの第{g}限月のデータサイズは{shape}"
        print(text)

    #　ナイトJNET
    df_night_JNET_mini = df_night[(df_night["銘柄"]=="日経225（ミニ）") & (df_night["先物/OP"] == "先物")]

    # 限月の抽出
    df_night_JNET_mini = get_gengetsu(df_night_JNET_mini)

    # サイズの確認
    for g in df_night_JNET_mini["限月"].unique():
        shape = df_night_JNET_mini[df_night_JNET_mini["限月"] == g].shape
        
        text = f"ナイト日経225ミニJNETの第{g}限月のデータサイズは{shape}"
        print(text)

    df_night_mini_stacked = make_long_df(df_night_mini)
    df_night_JNET_mini_stacked = make_long_df(df_night_JNET_mini)
    df_night_mini_sum = merge_JNET(df_night_mini_stacked, df_night_JNET_mini_stacked)

    # TOPIX
    #　ナイト立会
    df_night_topix = df_night[(df_night["銘柄"]=="TOPIX") & (df_night["先物/OP"] == "先物")]

    # 限月の抽出
    df_night_topix = get_gengetsu(df_night_topix)

    # サイズの確認
    for g in df_night_topix["限月"].unique():
        shape = df_night_topix[df_night_topix["限月"] == g].shape
        
        text = f"ナイトTOPIXの第{g}限月のデータサイズは{shape}"
        print(text)

    #　ナイトJNET
    df_night_JNET_topix = df_night[(df_night["銘柄"]=="TOPIX") & (df_night["先物/OP"] == "先物")]

    # 限月の抽出
    df_night_JNET_topix = get_gengetsu(df_night_JNET_topix)

    # サイズの確認
    for g in df_night_JNET_topix["限月"].unique():
        shape = df_night_JNET_topix[df_night_JNET_topix["限月"] == g].shape
        
        text = f"ナイトTOPIXのJNETの第{g}限月のデータサイズは{shape}"
        print(text)
    
    df_night_topix_stacked = make_long_df(df_night_topix)
    df_night_JNET_topix_stacked = make_long_df(df_night_JNET_topix)
    df_night_topix_sum = merge_JNET(df_night_topix_stacked, df_night_JNET_topix_stacked)


4. 立会とJNETを統合します
4-2. 日中取引の統合…
日中225ラージの第1限月のデータサイズは(20, 16)
日中225ラージの第2限月のデータサイズは(12, 16)
日中225ラージJNETの第1限月のデータサイズは(16, 16)
日中225ラージJNETの第2限月のデータサイズは(3, 16)
日中225ミニの第1限月のデータサイズは(18, 16)
日中225ミニの第2限月のデータサイズは(20, 16)
日中225ミニの第3限月のデータサイズは(9, 16)
日中225ミニJNETの第1限月のデータサイズは(3, 16)
日中225ミニJNETの第2限月のデータサイズは(3, 16)
日中225ミニJNETの第3限月のデータサイズは(1, 16)
日中TOPIXの第1限月のデータサイズは(20, 16)
日中TOPIX、JNETの第1限月のデータサイズは(16, 16)
4-3. ナイトセッションの統合…
ナイト日経225ラージの第1限月のデータサイズは(20, 16)
ナイト日経225ラージの第2限月のデータサイズは(10, 16)
ナイト日経225ラージJNETの第1限月のデータサイズは(7, 16)
ナイト日経225ラージJNETの第2限月のデータサイズは(1, 16)
ナイト日経225ミニの第1限月のデータサイズは(17, 16)
ナイト日経225ミニの第2限月のデータサイズは(20, 16)
ナイト日経225ミニの第3限月のデータサイズは(12, 16)
ナイト日経225ミニJNETの第1限月のデータサイズは(17, 16)
ナイト日経225ミニJNETの第2限月のデータサイズは(20, 16)
ナイト日経225ミニJNETの第3限月のデータサイズは(12, 16)
ナイトTOPIXの第1限月のデータサイズは(20, 16)
ナイトTOPIXのJNETの第1限月のデータサイズは(20, 16)


In [7]:
df_wholeday_large_sum.head()
#df_wholeday_mini_sum
#df_wholeday_topix_sum
#df_night_large_sum
#df_night_mini_sum
#df_night_topix_sum

,institutions_code,institutions,institutions_eng,volume,JPX_code,instrument,date,先物/OP,銘柄,限月_月,限月_年,限月,sell_buy
0,12479,ＡＢＮアムロクリア,ABN AMRO Clearing,11166,166030018,FUT_NK225_2103,2021-01-27,先物,日経225（ラージ）,3,2021,1,sell
1,11788,ソシエテ・ジェネラル,Societe Generale,7286,166030018,FUT_NK225_2103,2021-01-27,先物,日経225（ラージ）,3,2021,1,sell
2,11714,ＪＰモルガン証券,JPMorgan Japan,1259,166030018,FUT_NK225_2103,2021-01-27,先物,日経225（ラージ）,3,2021,1,sell
3,12400,野村證券,Nomura Securities,1178,166030018,FUT_NK225_2103,2021-01-27,先物,日経225（ラージ）,3,2021,1,sell
4,11256,ＳＢＩ証券,SBI SECURITIES,1367,166030018,FUT_NK225_2103,2021-01-27,先物,日経225（ラージ）,3,2021,1,sell


In [9]:
## 5. 銘柄ごとに日中取引の情報をナイトから補う
text = "\n5. 銘柄ごとに日中取引の情報をナイトから補います"
print(text)

### 5-2. 情報を補完する
# 日経225ラージ
df_wholeday_large_sum_full = complement_night(df_day=df_wholeday_large_sum, df_night=df_night_large_sum)

# 日経225ミニ
df_wholeday_mini_sum_full = complement_night(df_day=df_wholeday_mini_sum, df_night=df_night_mini_sum)
# 日経225ミニの値を1/10する
df_wholeday_mini_sum_full["volume"] = df_wholeday_mini_sum_full["volume"] / 10

# TOPIX
df_wholeday_topix_sum_full = complement_night(df_day=df_wholeday_topix_sum, df_night=df_night_topix_sum)
print("df_wholeday_topix_sum_full colnames", df_wholeday_topix_sum_full.columns)

## 6. 銘柄別で全限月の合計を出す
text = "\n6. 銘柄別で全限月の合計を計算"
print(text)


5. 銘柄ごとに日中取引の情報をナイトから補います
補完前の日中データサイズ： (63, 13)
欠損データなし
欠損データなし
欠損データなし
欠損データなし
補完後の日中データサイズ： (63, 13)
欠損データなし
欠損データなし
欠損データなし
欠損データなし
補完後の日中データサイズ： (63, 13)
欠損データなし
欠損データなし
会社コード11714の第2限月にはsellがない
会社コード11714の第2限月のsellを3で補完した
補完後の日中データサイズ： (64, 13)
欠損データなし
欠損データなし
補完後の日中データサイズ： (64, 13)
欠損データなし
欠損データなし
欠損データなし
欠損データなし
補完後の日中データサイズ： (64, 13)
欠損データなし
欠損データなし
補完後の日中データサイズ： (64, 13)
欠損データなし
欠損データなし
欠損データなし
欠損データなし
補完後の日中データサイズ： (64, 13)
会社コード12328の第1限月にはbuyがない
会社コード12328の第1限月のbuyを439で補完した
欠損データなし
欠損データなし
補完後の日中データサイズ： (65, 13)
欠損データなし
欠損データなし
欠損データなし
欠損データなし
補完後の日中データサイズ： (65, 13)
欠損データなし
欠損データなし
欠損データなし
欠損データなし
補完後の日中データサイズ： (65, 13)
欠損データなし
会社コード11635の第2限月にはbuyがない
ナイト・セッションには該当データなし
欠損データなし
補完後の日中データサイズ： (65, 13)
欠損データなし
欠損データなし
補完後の日中データサイズ： (65, 13)
欠損データなし
欠損データなし
補完後の日中データサイズ： (65, 13)
会社コード12176の第1限月にはbuyがない
ナイト・セッションには該当データなし
補完後の日中データサイズ： (65, 13)
欠損データなし
欠損データなし
補完後の日中データサイズ： (65, 13)
欠損データなし
欠損データなし
会社コード11060の第2限月にはsellがない
会社コード11060の第2限月のsellを10で補完した
補完後の日中データサイズ： (66, 13)


# modifications

In [10]:
df_wide_total = pd.concat([df_wholeday_large_sum_full, df_wholeday_mini_sum_full, df_wholeday_topix_sum_full])
df_wide_total.head()

,institutions_code,institutions,institutions_eng,volume,JPX_code,instrument,date,先物/OP,銘柄,限月_月,限月_年,限月,sell_buy
0,12479,ＡＢＮアムロクリア,ABN AMRO Clearing,11166.0,166030018,FUT_NK225_2103,2021-01-27,先物,日経225（ラージ）,3,2021,1,sell
1,11788,ソシエテ・ジェネラル,Societe Generale,7286.0,166030018,FUT_NK225_2103,2021-01-27,先物,日経225（ラージ）,3,2021,1,sell
2,11714,ＪＰモルガン証券,JPMorgan Japan,1259.0,166030018,FUT_NK225_2103,2021-01-27,先物,日経225（ラージ）,3,2021,1,sell
3,12400,野村證券,Nomura Securities,1178.0,166030018,FUT_NK225_2103,2021-01-27,先物,日経225（ラージ）,3,2021,1,sell
4,11256,ＳＢＩ証券,SBI SECURITIES,1367.0,166030018,FUT_NK225_2103,2021-01-27,先物,日経225（ラージ）,3,2021,1,sell


In [11]:
# Appending groups
df_group = pd.read_excel("企業名グループ.xlsx", converters={'group':str,'institutions_code':str})
df_wide_total = pd.merge(df_wide_total, df_group, how='left')

In [14]:
### 6-1. 銘柄ごとに売り・買いを横並びにして差を計算する
# ラージ
df_wholeday_large_total = df_wholeday_large_sum_full.groupby(["institutions_code", 
                                                            "sell_buy"]).agg({"institutions":max,
                                                                            "institutions_eng":max,
                                                                            "JPX_code":max,
                                                                            "instrument":max,
                                                                            "date":max,
                                                                            "先物/OP":max,
                                                                            "銘柄":max,
                                                                            "volume":sum}).reset_index()
df_large_pivoted = df_wholeday_large_total.pivot(index='institutions_eng', columns='sell_buy', values='volume')
df_large_pivoted = df_large_pivoted.reset_index().rename_axis(None, axis=1)
df_large_pivoted["diff"] = df_large_pivoted["buy"] - df_large_pivoted["sell"]
df_large_pivoted["product"] = df_wholeday_large_total['銘柄'].unique()[0] # modification
df_large_pivoted["date"] = day.strftime("%Y-%m-%d")

# ミニ
df_wholeday_mini_total = df_wholeday_mini_sum_full.groupby(["institutions_code", 
                                                        "sell_buy"]).agg({"institutions":max,
                                                                            "institutions_eng":max,
                                                                            "JPX_code":max,
                                                                            "instrument":max,
                                                                            "date":max,
                                                                            "先物/OP":max,
                                                                            "銘柄":max,
                                                                            "volume":sum}).reset_index()
df_mini_pivoted = df_wholeday_mini_total.pivot(index='institutions_eng', columns='sell_buy', values='volume')
df_mini_pivoted = df_mini_pivoted.reset_index().rename_axis(None, axis=1)
df_mini_pivoted["diff"] = df_mini_pivoted["buy"] - df_mini_pivoted["sell"]   
df_mini_pivoted["product"] = df_wholeday_mini_total['銘柄'].unique()[0] # modification    
df_mini_pivoted["date"] = day.strftime("%Y-%m-%d")

# TOPIX
df_wholeday_topix_total = df_wholeday_topix_sum_full.groupby(["institutions_code", 
                                                            "sell_buy"]).agg({"institutions":max,
                                                                            "institutions_eng":max,
                                                                            "JPX_code":max,
                                                                            "instrument":max,
                                                                            "date":max,
                                                                            "先物/OP":max,
                                                                            "銘柄":max,
                                                                            "volume":sum}).reset_index()

df_topix_pivoted = df_wholeday_topix_total.pivot(index='institutions_eng', columns='sell_buy', values='volume')
df_topix_pivoted = df_topix_pivoted.reset_index().rename_axis(None, axis=1)
df_topix_pivoted["diff"] = df_topix_pivoted["buy"] - df_topix_pivoted["sell"]
df_topix_pivoted["product"] = df_wholeday_topix_total['銘柄'].unique()[0] # modification
df_topix_pivoted["date"] = day.strftime("%Y-%m-%d")

In [15]:
df_large_pivoted.head()

,institutions_eng,buy,sell,diff,product,date
0,ABN AMRO Clearing,9666.0,11195.0,-1529.0,日経225（ラージ）,2021-01-27
1,BNP Paribas (Japan),NaN,1001.0,NaN,日経225（ラージ）,2021-01-27
2,Barclays Japan,576.0,750.0,-174.0,日経225（ラージ）,2021-01-27
3,BofA Securities,867.0,1091.0,-224.0,日経225（ラージ）,2021-01-27
4,Citigroup Japan,395.0,77.0,318.0,日経225（ラージ）,2021-01-27


In [35]:
### 6-2. 銘柄別のデータフレームを横並びに統合する      
df_total_final = pd.concat([dfi.set_index('institutions_eng') for dfi in [df_large_pivoted, df_mini_pivoted, df_topix_pivoted]], 
                        axis=0, # modification
                        sort=False).reset_index()
df_total_final.head()

,institutions_eng,buy,sell,diff,product,date
0,ABN AMRO Clearing,9666.0,11195.0,-1529.0,日経225（ラージ）,2021-01-27
1,BNP Paribas (Japan),NaN,1001.0,NaN,日経225（ラージ）,2021-01-27
2,Barclays Japan,576.0,750.0,-174.0,日経225（ラージ）,2021-01-27
3,BofA Securities,867.0,1091.0,-224.0,日経225（ラージ）,2021-01-27
4,Citigroup Japan,395.0,77.0,318.0,日経225（ラージ）,2021-01-27


In [36]:
# 日経ラージとミニの合計を元DFに足す
df_nikkei_sum = df_total_final[df_total_final["product"]!="TOPIX"].groupby('institutions_eng').agg({"buy":sum, "sell":sum, "diff":sum, "date":max}).reset_index()
df_nikkei_sum['product'] = "Nikkei sum"
df_total_final = pd.concat([df_total_final, df_nikkei_sum])

In [37]:
# 総合計を元DFに足す
def multiply_TOPIX(number): 
    return TOPIX_MULTI * number 

df_topix = df_total_final[df_total_final['product']=='TOPIX']
df_topix[["buy", "sell", "diff"]] = df_topix[["buy", "sell", "diff"]].apply(multiply_TOPIX)
df_topix['product'] = "TOPIX adjusted"

df_sum = pd.concat([df_nikkei_sum, df_topix])

df_sum = df_sum.groupby('institutions_eng').agg({"buy":sum, "sell":sum, "diff":sum, "date":max}).reset_index()
df_sum["product"] = "total"

df_total_final = pd.concat([df_total_final, df_sum])

In [38]:
df_total_final = pd.merge(df_total_final, df_group, how='left')

In [39]:
df_total_final.head()

,buy,date,diff,institutions_eng,product,sell,institutions,group,institutions_code
0,9666.0,2021-01-27,-1529.0,ABN AMRO Clearing,日経225（ラージ）,11195.0,ＡＢＮアムロクリア,2,12479
1,NaN,2021-01-27,NaN,BNP Paribas (Japan),日経225（ラージ）,1001.0,ＢＮＰパリバ証券,2,12428
2,576.0,2021-01-27,-174.0,Barclays Japan,日経225（ラージ）,750.0,バークレイズ証券,2,12410
3,867.0,2021-01-27,-224.0,BofA Securities,日経225（ラージ）,1091.0,BofA証券,1,12792
4,395.0,2021-01-27,318.0,Citigroup Japan,日経225（ラージ）,77.0,シティグループ証券,1,11792


# 変更終わり（１月２１日）

In [40]:
df_by_group_product = df_total_final.groupby(["group", "product"]).agg({"buy":sum, "sell":sum, "diff":sum, "date":max}).reset_index()
#df_by_group_product["date"] = day.strftime("%Y-%m-%d")
df_by_group_product.head()

,group,product,buy,sell,diff,date
0,1,Nikkei sum,4793.4,3540.3,1253.1,2021-01-27
1,1,TOPIX,9143.0,9342.0,-199.0,2021-01-27
2,1,total,13936.4,12882.3,1054.1,2021-01-27
3,1,日経225（ミニ）,831.4,575.3,256.1,2021-01-27
4,1,日経225（ラージ）,3962.0,2965.0,997.0,2021-01-27


In [41]:
df_by_group = df_total_final.groupby(["group"]).agg({"buy":sum, "sell":sum, "diff":sum, "date":max}).reset_index()
#df_by_group["date"] = day.strftime("%Y-%m-%d")
df_by_group.head()

,group,buy,sell,diff,date
0,1,32666.2,29304.9,3361.3,2021-01-27
1,2,182167.8,192687.0,-6724.2,2021-01-27
2,3,25552.1,22245.0,2389.1,2021-01-27
3,4,40734.6,40584.2,155.2,2021-01-27


In [42]:
# appending historical data
try:
    historical_by_group = pd.read_csv("historical_by_group.csv")
    historical_by_group = pd.concat([historical_by_group, df_by_group])
except:
    historical_by_group = df_by_group
historical_by_group.to_csv("historical_by_group.csv")

In [43]:
# appending historical data
try:
    historical_by_group_product = pd.read_csv("historical_by_group_product.csv")
    historical_by_group_product = pd.concat([historical_by_group_product, df_by_group_product])
except:
    historical_by_group_product = df_by_group_product
historical_by_group_product.to_csv("historical_by_group_product.csv")